In [58]:
import os
import numpy as np
import pandas as pd
from collections import defaultdict
from tqdm import tqdm

In [59]:
def get_all_korean():

    def nextKorLetterFrom(letter):
        lastLetterInt = 15572643
        if not letter:
            return '가'
        a = letter
        b = a.encode('utf8')
        c = int(b.hex(), 16)

        if c == lastLetterInt:
            return False

        d = hex(c + 1)
        e = bytearray.fromhex(d[2:])

        flag = True
        while flag:
            try:
                r = e.decode('utf-8')
                flag = False
            except UnicodeDecodeError:
                c = c+1
                d = hex(c)
                e = bytearray.fromhex(d[2:])
        return e.decode()

    returns = []
    flag = True
    k = ''
    while flag:
        k = nextKorLetterFrom(k)
        if k is False:
            flag = False
        else:
            returns.append(k)
    return returns

In [60]:
def korean2label(letter):
    ch1 = (ord(letter) - ord('가'))//588
    ch2 = ((ord(letter) - ord('가')) - (588*ch1)) // 28
    ch3 = (ord(letter) - ord('가')) - (588*ch1) - 28*ch2
    return np.array([ch1,ch2,ch3])

In [61]:
recieved_letters = "갊갸곁곬교궤껴꽤꾜끝높뉑늬닠닦덤돛돼땍뛰뜻라럽롵뤄많몇묟무벚붉뷔뺨뽱쁘삯셴숲쉽쎄쏟쑉씩얹없왼읊읨잃쟨점좋줬짚짧쫄쯷찧찾쳇춰츄치캐캠쾅쿵탤턔토튠특팼퍗폐푀퓸핥흡횃훨"

In [62]:
recieved_letters_labels = [korean2label(l) for l in recieved_letters]

In [63]:
ak = get_all_korean()

In [64]:
akl = [korean2label(k) for k in ak]

In [65]:
# recieved_letters에서 초중종성중 최소 하나가 같은 글자 + 초중종성중 둘이 같은 글자

In [66]:
akmap = defaultdict(set)

In [67]:
pbar = tqdm(ak)
akmap = defaultdict(set)
for l in pbar:
    labels = korean2label(l)
    for rl, rlabel in zip(recieved_letters, recieved_letters_labels):
        match = labels==rlabel
        if np.sum(match)>=2:
            akmap[l].add(rl)
    # for al,alabel in zip(ak,akl):
    #     if np.sum(labels==alabel) >= 2:
    #         akmap[l].add(al)
    pbar.set_postfix(a=l, m=akmap[l])

100%|██████████| 11172/11172 [00:11<00:00, 1001.88it/s, a=힣, m={'찧'}]                        


In [70]:
pbar = tqdm(ak)
for l in pbar:
    if len(akmap[l]) == 0:
        label = korean2label(l)
        for rl, rlabel in zip(recieved_letters, recieved_letters_labels):
            match = labels==rlabel
            if label[0]==0:
                if match[0] or match[1]:
                    akmap[l].add(rl)
            else:
                if np.sum(match) >= 1:
                    akmap[l].add(rl)

100%|██████████| 11172/11172 [00:00<00:00, 439996.66it/s]


In [69]:
pbar = tqdm(ak)
for l in pbar:
    try:
        akmap[l].remove(l)
    except:
        pass

100%|██████████| 11172/11172 [00:00<00:00, 1221329.90it/s]


In [71]:
akmap

defaultdict(set,
            {'가': {'갊', '갸', '교', '궤', '라'},
             '각': {'갊'},
             '갂': {'갊', '닦'},
             '갃': {'갊', '삯'},
             '간': {'갊'},
             '갅': {'갊'},
             '갆': {'갊', '많'},
             '갇': {'갊'},
             '갈': {'갊'},
             '갉': {'갊'},
             '갊': {'닠', '씩', '잃', '짚', '찧', '치', '핥', '횃', '훨', '흡'},
             '갋': {'갊', '짧'},
             '갌': {'갊', '곬'},
             '갍': {'갊', '핥'},
             '갎': {'갊'},
             '갏': {'갊'},
             '감': {'갊'},
             '갑': {'갊'},
             '값': {'갊'},
             '갓': {'갊'},
             '갔': {'갊'},
             '강': {'갊'},
             '갖': {'갊', '찾'},
             '갗': {'갊'},
             '갘': {'갊'},
             '같': {'갊', '곁'},
             '갚': {'갊'},
             '갛': {'갊'},
             '개': {'갸', '교', '궤', '캐'},
             '객': {'땍'},
             '갞': {'닠', '씩', '잃', '짚', '찧', '치', '핥', '횃', '훨', '흡'},
             '갟': {'닠', '씩', '잃', '짚', '찧',

In [73]:
import copy
bkmap = copy.deepcopy(akmap)
pbar = tqdm(ak)
for l in pbar:
    labels = korean2label(l)
    for al,alabel in zip(ak,akl):
        if np.sum(labels==alabel) >= 2:
            bkmap[l].add(al)

100%|██████████| 11172/11172 [08:41<00:00, 21.41it/s]


In [ ]:
pbar = tqdm(ak)
for l in pbar:
    try:
        bkmap[l].remove(l)
    except:
        pass

In [80]:
aklist = [{"target":l, "similar":sorted(list(akmap[l]))} for l in ak]
akdf = pd.DataFrame().from_dict(aklist)
adf = pd.DataFrame(index=akdf.target,data={'similar':akdf.similar.values})
adf.to_pickle("/home/jupyter/ai_font/data/pickle/letter_mapper_a.pickle")

In [81]:
bklist = [{"target":l, "similar":sorted(list(bkmap[l]))} for l in ak]
bkdf = pd.DataFrame().from_dict(bklist)
bdf = pd.DataFrame(index=bkdf.target,data={'similar':bkdf.similar.values})
bdf.to_pickle("/home/jupyter/ai_font/data/pickle/letter_mapper_b.pickle")

In [77]:
adf = pd.DataFrame(index=akdf.target,data={'similar':akdf.similar.values})

In [82]:
bdf

,similar
target,
가,"[가, 각, 갂, 갃, 간, 갅, 갆, 갇, 갈, 갉, 갊, 갋, 갌, 갍, 갎, ..."
각,"[가, 각, 갂, 갃, 간, 갅, 갆, 갇, 갈, 갉, 갊, 갋, 갌, 갍, 갎, ..."
갂,"[가, 각, 갂, 갃, 간, 갅, 갆, 갇, 갈, 갉, 갊, 갋, 갌, 갍, 갎, ..."
갃,"[가, 각, 갂, 갃, 간, 갅, 갆, 갇, 갈, 갉, 갊, 갋, 갌, 갍, 갎, ..."
간,"[가, 각, 갂, 갃, 간, 갅, 갆, 갇, 갈, 갉, 갊, 갋, 갌, 갍, 갎, ..."
...,...
힟,"[깇, 낓, 닟, 닠, 딫, 띷, 맃, 및, 빛, 삧, 싳, 씩, 씿, 잃, 잋, ..."
힠,"[깈, 낔, 닠, 딬, 띸, 맄, 밐, 빜, 삨, 싴, 앀, 잌, 짘, 찤, 칰, ..."
힡,"[깉, 낕, 닠, 닡, 딭, 띹, 맅, 밑, 빝, 삩, 싵, 씩, 앁, 잃, 잍, ..."


In [28]:
adf.to_pickle("/home/jupyter/ai_font/data/pickle/letter_mapper.pickle")

In [37]:
path = '/home/jupyter/ai_font/data'
fonts = sorted([f.replace(".ttf","") for f in os.listdir(f"{path}/train_ttfs")])

In [83]:
fonts = sorted([(f.replace(".ttf",""), f"{path}/train_ttfs/{f}") for f in os.listdir(f"{path}/train_ttfs")])

In [86]:
font_letter = defaultdict(list)
for font,_ in tqdm(fonts):
    for k in ak:
        if os.path.exists(f"{path}/train/pngs/{font}__{k}.png"):
            font_letter[font].append(k)

100%|██████████| 413/413 [00:22<00:00, 18.65it/s]


In [90]:
font_mapper = pd.Series(font_letter)

In [92]:
font_mapper.to_pickle("/home/jupyter/ai_font/data/pickle/font_mapper.pickle")

In [88]:
[(k,len(v)) for k,v in font_letter.items()]

[('교보 손글씨 2019', 11172),
 ('교보 손글씨 2020 박도연', 11172),
 ('교보 손글씨 2021 성지영', 11172),
 ('교보 손글씨 2022 김혜남', 11172),
 ('교보 손글씨 2023 우선아', 11172),
 ('나눔손글씨 가람연꽃', 11172),
 ('나눔손글씨 갈맷글', 11172),
 ('나눔손글씨 강부장님체', 11172),
 ('나눔손글씨 강인한 위로', 11172),
 ('나눔손글씨 고딕 아니고 고딩', 11172),
 ('나눔손글씨 고려글꼴', 11172),
 ('나눔손글씨 곰신체', 11172),
 ('나눔손글씨 규리의 일기', 11172),
 ('나눔손글씨 금은보화', 11172),
 ('나눔손글씨 기쁨밝음', 11172),
 ('나눔손글씨 김유이체', 11172),
 ('나눔손글씨 꽃내음', 11172),
 ('나눔손글씨 끄트머리체', 11172),
 ('나눔손글씨 나는 이겨낸다', 11172),
 ('나눔손글씨 나무정원', 11172),
 ('나눔손글씨 나의 아내 손글씨', 11172),
 ('나눔손글씨 노력하는 동희', 11172),
 ('나눔손글씨 느릿느릿체', 11172),
 ('나눔손글씨 다시 시작해', 11172),
 ('나눔손글씨 다진체', 11172),
 ('나눔손ᄀ

In [97]:
import random

In [99]:
random.choice(font_mapper.loc[font_mapper.index[0]])

'쒆'

In [100]:
contents = font_mapper.loc[font_mapper.index[0]]

In [102]:
contents.pop(1)

'각'

In [110]:
random.choice(contents)

'쬻'

In [118]:
random.randint(0,1)

0